In [1]:
# Setting up the environment

import svelte_widget
import xgboost
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

/home/nicolazucchia/miniconda3/envs/svelte-widget/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/nicolazucchia/miniconda3/envs/svelte-widget/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit fo

2023-05-25 14:24:47.620402: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 14:24:47.667915: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 14:24:47.669106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 14:24:48.410006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Upload and set up data

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

df = pd.read_csv("BRCA.csv")

# Deal with non-numerical variables

df = df.drop(df.columns[0], axis=1)
df['Gender'] = df['Gender'].map({"FEMALE": 0, "MALE": 1})
df['Tumour_Stage'] = df['Tumour_Stage'].map({"I": 0, "II": 1, "III": 2})
df['Histology'] = df['Histology'].map({"Infiltrating Ductal Carcinoma": 0, "Mucinous Carcinoma": 1})
df['ER status'] = df['ER status'].map({"Negative": 0, "Positive": 1})
df['PR status'] = df['PR status'].map({'Negative': 0, 'Positive': 1})
df['HER2 status'] = df['HER2 status'].map({"Negative": 0, "Positive": 1})
df['Surgery_type'] = df['Surgery_type'].map({"Modified Radical Mastectomy": 0, "Lumpectomy": 1, "Simple Mastectomy": 2})
df['Patient_Status'] = df['Patient_Status'].map({"Dead": 0, "Alive": 1})
# Temporal variables for now are not considered
'''
for i in range(0, df.shape[0]):
    df.loc[i, 'Date_of_Surgery'] = datetime.strptime(df.loc[i, 'Date_of_Surgery'], '%d-%b-%y')
    df.loc[i, 'Date_of_Last_Visit'] = datetime.strptime(df.loc[i, 'Date_of_Last_Visit'], '%d-%b-%y')
'''
df = df.drop(['Date_of_Surgery', 'Date_of_Last_Visit'], axis=1)

df = df.dropna()

# Initialize the scaler
scaler = MinMaxScaler()

# Apply the scaler to every column in the dataframe
stdDf = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

stdX = stdDf.drop(columns=['Patient_Status'])

# We do not want to scale Y, so widget will display real values
# for predictions and not scaled ones
stdY = stdDf['Patient_Status']

In [3]:
df

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Patient_Status
0,36,0,0.080353,0.42638,0.547150,0.273680,2,0.0,1,1,0,0.0,1.0
1,43,0,-0.420320,0.57807,0.614470,-0.031505,1,1.0,1,1,0,1.0,0.0
3,56,0,0.345090,-0.21147,-0.193040,0.124270,1,0.0,1,1,0,0.0,1.0
5,84,1,-0.081872,1.72410,-0.057335,0.043025,2,0.0,1,1,0,0.0,1.0
6,53,0,-0.069535,1.41830,-0.361050,0.391580,1,0.0,1,1,0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,60,0,0.094585,1.00030,-0.225750,0.048602,2,0.0,1,1,0,2.0,0.0
329,36,0,0.231800,0.61804,-0.557790,-0.517350,2,0.0,1,1,1,2.0,0.0
331,61,0,-0.719470,2.54850,-0.150240,0.339680,1,0.0,1,1,0,1.0,0.0
332,79,0,0.479400,2.05590,-0.531360,-0.188480,0,0.0,1,1,1,1.0,0.0


In [4]:
# Defining and training the two models

from sklearn.linear_model import LinearRegression

stdX_train, stdX_test, stdy_train, stdy_test = train_test_split(stdX, stdY, test_size=0.2, random_state=42)

# Model1
stdmodel1 = xgboost.XGBRegressor(random_state=42).fit(stdX_train,stdy_train)

# Model2
stdmodel2 = LinearRegression().fit(stdX_train, stdy_train)

In [5]:
z = svelte_widget.ExampleWidget(stdX_test,stdy_test,stdmodel1.predict,stdmodel2.predict)
z

Start...
Predictions obtained
Shaps now
Shaps XGB
Shaps NN
Calculating...
-> m1
-> m2
Difference...
Done


ExampleWidget(dataset={'Age': [0.13793103448275867, 0.6724137931034482, 0.5862068965517242, 0.5344827586206897…